In [11]:
!pip install pandas | wc -l
!pip install tensorflow | wc -l
!pip install tensorflow-gpu | wc -l

5
43
43


In [12]:
pwd()

'\\\\wsl.localhost\\Ubuntu-20.04\\home\\mdc20\\repos\\personal\\project-fashion-finder\\model\\src'

In [13]:
CLASS_LIST_14 = ["Tee","Tank","Dress","Shorts","Skirt","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Romper","Sweatpants","Jacket"]
CLASS_LIST_29 = ["Tee","Tank","Dress","Shorts","Skirt","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Romper","Sweatpants","Jacket", "Jeans", "Poncho", "Button-Down", "Pencil", "Maxi",  "Floral", "Pleated", "Mesh","Bandana", "Tie-Dye", "Culottes", "Embroidered", "Kimono","Chevron", "Buttoned"]
CLASS_LIST_35 = ["Tee","Tank","Dress","Shirt","Shorts","Skirt","Hoodie","Jumpsuit","Cargo","Turtleneck","Sweater","Plaid","Blazer","Striped","Cardigan","Blouse","Romper","Jacket","Jeans","Maxi","Floral","Denim","Trench","Baroque","Ornate","Belted","Jersey","Sweatshorts","PJ","Parka","Polka","Beaded","Shawl","Bodycon","Abstract"]
CLASS_LIST_21 = ["Tee","Tank","Dress","Shorts","Skirt","Hoodie","Jumpsuit","Sweater","Blazer","Striped","Cardigan","Blouse","Jacket","Jeans","Maxi","Floral","Denim","Sweatshorts","Polka","Shawl","Bodycon"]
CLASS_LIST_2 = ["Dress", "Jeans"]
CLASS_LIST = CLASS_LIST_21

In [14]:
import os
import numpy as np
import scipy
import PIL
from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS = None
import tensorflow.keras as keras
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [15]:
from collections import defaultdict
import pandas as pd
datasetdir = 'C:\\Users\\mdc20\\Downloads\\img'
os.chdir(datasetdir)
def constructImageClassDataFrame(shape, min_class_occurence = 1, class_list = CLASS_LIST):
    # Args: 
    #   Shape: Image shape (2D)
    #   min_class_occurence: number of times class must occur in labeled dir in order to add it to the final sclass list
    
    
    sub_dirs = [d for d in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(datasetdir, d))]
    
    if class_list is None:
        classes = defaultdict(lambda: 0)

        for sub_dir in sub_dirs:
            labels = sub_dir.split('_')
            for label in labels:
                classes[label] += 1
        top_k_classes = [cls for cls in classes if classes[cls] >= min_class_occurence]
        label_classes = top_k_classes    
    else:
        classes = class_list
        label_classes = classes

    
    # classes = list(classes)
    # print("Done calculating list of classes")

    # TODO: remove
    # top_k_classes = ['Graphic', 'Tee', 'Tank', 'Dress', 'Shorts', 'Print', 'Skirt', 'Blouse', 'Top', 'Leggings', 'Sweater', 'Abstract', 'Romper', 'Jumpsuit', 'Floral', 'Cardigan',  'Stripe', 'Boxy','Joggers', 'Striped',  'Tribal', 'Jacket', 'Jeans' ]
    # top_k_classes = ['Top', 'Blouse']

    arr = [[labeled_dir, file] for labeled_dir in sub_dirs for file in os.listdir(labeled_dir) ]
    print("Done parsing through directories")
    df = pd.DataFrame(data=arr, columns=["folder","filename"])



    df['filename'] = df['folder'] + '/' + df['filename']
    df['labels'] = df['folder'].apply(lambda x : [y for y in x.split('_') if y in label_classes] if len([y for y in x.split('_') if y in label_classes]) > 0 else None)

    print(np.shape(df))
    # index_names = df [df['labels'] == []].index
    # Drop Dataframe if none of its labels are present in the selected classes
    df = df[df.labels.notnull()]

    # series_list = list()
    # print("Creating OHE for classes")
    # for idx, cls in enumerate(classes):
    #     if idx %50 == 0: print(idx)
    #     series = df['filename'].apply(lambda x: 1 if cls in x else 0)
    #     series_list.append(series)
    #     # df[str(cls)] = 1 if df['folder'].str.contains(str(cls)) else 0
    
    # df = pd.concat([df, series_list], axis=1)

    #     files = os.listdir(labeled_dir)
    #     labels = labeled_dir.split("_")
    #     for file in files:
    #         continue
            # df.loc[len(df.index)] = [os.path.join(datasetdir, labeled_dir), labels]

    
    # df.head()

    return df

# df, top_k_classes = constructImageClassDataFrame(shape = (256,256), min_class_occurence=10,class_list=['Shirt','Shorts'])
# print(df.head(100))
# print(np.shape(df))
# print(top_k_classes)
# print(np.shape(top_k_classes))
    

In [16]:
datasetdir = 'E:\\img_highres'
os.chdir(datasetdir)

from msilib.schema import Directory
from random import shuffle
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 64
IMG_SHAPE = (224,224)


def DataLoad(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1,
        rescale = 1.0/255, 
    )

    height, width = shape

    df = constructImageClassDataFrame(shape, class_list=CLASS_LIST)

    print(df.sample(n=10))
    # Modify dataframe to only take images with references to classes

    print("Num classes #")
    print(len(CLASS_LIST)) 
    # for subdir

    train_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        target_size = IMG_SHAPE,
        seed = 42,
        shuffle = True,
        batch_size = BATCH_SIZE,
        class_mode="categorical",
        classes = CLASS_LIST,
        subset = 'training'
    )
    val_dataset = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = datasetdir,
        x_col="filename",
        y_col="labels",
        target_size = IMG_SHAPE,
        seed = 42,
        shuffle = True,
        batch_size = BATCH_SIZE,
        class_mode="categorical",
        classes = CLASS_LIST,
        subset = 'validation'
    )


    return train_dataset, val_dataset, 

# train_dataset, val_dataset = DataLoad((350,350), preprocessing=preprocess_input)


In [17]:
import scipy
from keras.applications.vgg16 import VGG16, preprocess_input

# Create custom preproocessing routine


train_dataset, val_dataset = DataLoad((224,224), preprocessing=preprocess_input)

print(type(train_dataset))
print(type(val_dataset))
# print(train_dataset.shape())

X_train, y_train = next(train_dataset)

print(type(X_train))
print(y_train[0])
print(np.sum(y_train, axis=1))
print(np.sum(y_train, axis=0))
print(np.shape(X_train))
print(np.shape(y_train))
# print(y_train.shape())    

Done parsing through directories
(289212, 3)
                                 folder  \
165076  Leopard_Print_Longline_Cardigan   
201983                Plaid_Shirt_Dress   
228499          Shore_Girl_Cropped_Tank   
74179                 Crochet_Mesh_Tank   
180548        Metallic_Striped_Knit_Tee   
146280       High-Waisted_Lounge_Shorts   
263887        Tie-Dye_Longline_Cardigan   
78477    Cutout-Back_Floral_Print_Dress   
82698        Daydreamer_Crocheted_Dress   
137284            Glitter_Bodycon_Dress   

                                                filename            labels  
165076  Leopard_Print_Longline_Cardigan/img_00000040.jpg        [Cardigan]  
201983                Plaid_Shirt_Dress/img_00000056.jpg           [Dress]  
228499          Shore_Girl_Cropped_Tank/img_00000040.jpg            [Tank]  
74179                 Crochet_Mesh_Tank/img_00000018.jpg            [Tank]  
180548        Metallic_Striped_Knit_Tee/img_00000009.jpg    [Striped, Tee]  
146280       High-W

In [18]:
y_peek = y_train[:5]

In [19]:
vgg16 = keras.applications.vgg16
conv_model = vgg16.VGG16(weights='imagenet', include_top=False)
# conv_model.summary()


In [20]:
conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in conv_model.layers:
    layer.trainable = False

# flatten the output of the convolutional part:
# x = keras.layers.Dropout(0.25)(conv_model.output)
 
x = keras.layers.Flatten()(conv_model.output)
# three hidden layers}
x = keras.layers.Dense(32, activation='relu')(x)
# x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(32, activation='relu')(x)
# x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(32, activation='relu')(x)
# x = keras.layers.Dropout(0.5)(x)

# x = keras.layers.Dense(64, activation='relu')(x)
# final softmax layer with 40 categories
predictions = keras.layers.Dense(len(CLASS_LIST), activation = 'sigmoid')(x)
# creating the full model:
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [21]:
# Configure run environment
from tensorflow.python.client import device_lib

device_lib.list_local_devices()
# tf.config.list_physical_devices('CPU')

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8790317263064311798
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5762973696
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15975444588622002591
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

# Categorical Cross Entropy

## Weighted Binary Cross-Entropy Loss

In [1]:
import tensorflow as tf
import keras.backend as tfb
import numpy as np

POS_WEIGHT = 2  # multiplier for positive targets, needs to be tuned

def weighted_binary_crossentropy(target, output):
    """
    Weighted binary crossentropy between an output tensor 
    and a target tensor. POS_WEIGHT is used as a multiplier 
    for the positive targets.

    Combination of the following functions:
    * keras.losses.binary_crossentropy
    * keras.backend.tensorflow_backend.binary_crossentropy
    * tf.nn.weighted_cross_entropy_with_logits
    """
    # transform back to logits
    _epsilon = tfb._to_tensor(tfb.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
    output = tfb.log(output / (1 - output))
    # compute weighted loss
    loss = tf.nn.weighted_cross_entropy_with_logits(labels=target,
                                                    logits=output,
                                                    pos_weight=POS_WEIGHT)
    return tf.reduce_mean(loss, axis=-1)

### F1-Metric Loss

In [23]:
import tensorflow.keras.backend as K
import tensorflow as tf
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [24]:
from keras.callbacks import ModelCheckpoint
import time
histories = list()
for learning_rate in [0.001]:

    checkpoint = ModelCheckpoint("C:\\Users\mdc20\\best_model_21_class_lr@" + str(learning_rate) + "-" + str(time.time()) + ".hdf5", monitor='loss', verbose=1,
        save_best_only=True, mode='auto', period=1)
# from tf.keras.losses import BinaryCrossentropy

    # cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM, from_logits=False)
    # bce = tf.keras.losses.binary_crossentropy

    # bce = BinaryCrossentropy()
    full_model.compile(loss='weighted_binary_crossentropy',
                    optimizer=keras.optimizers.Adamax(learning_rate=learning_rate, beta_1 = 0.9, beta_2=0.99),
                    metrics=['acc', tf.keras.metrics.CategoricalCrossentropy()],
                    )
    history = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=2,
    epochs=50,
    steps_per_epoch = 1500,
    validation_steps = 400,
    callbacks=[checkpoint]
    )
    histories.append(history)

print(type(histories))

Epoch 1/50
 418/1500 [=======>......................] - ETA: 4:45 - loss: 0.4073 - acc: 0.3830 - categorical_crossentropy: 3.0164

KeyboardInterrupt: 

## Restart training from kernel

### Define saving routine

In [2]:
LEARNING_RATE = 0.002


In [3]:
from keras.callbacks import ModelCheckpoint
import time

checkpoint = ModelCheckpoint("C:\\Users\mdc20\\best_model_21_class_lr@" + str(LEARNING_RATE) + "-" + str(time.time()) + ".hdf5", monitor='loss', verbose=1,
        save_best_only=True, mode='auto', period=1)

In [27]:
# Restart training
EPOCH = 50

MODEL_FOLDER_PATH = 'C:\\Users\\mdc20\\'
MODEL_FILE = 'best_model_21_class_lr@0.001-1666709152.5078835.hdf5'
MODEL_FILE = 'best_model_21_class_lr@0.002-1666761586.9275644.hdf5'

restored_model = tf.keras.models.load_model(MODEL_FOLDER_PATH + MODEL_FILE, custom_objects={'weighted_binary_crossentropy': weighted_binary_crossentropy})
# restored_model.add_loss()
restored_model.compile(loss='binary_crossentropy',
                    optimizer=keras.optimizers.Adamax(learning_rate=LEARNING_RATE, beta_1 = 0.9, beta_2=0.99),
                    metrics=['acc'],
                    )
# tf.keras.backend.set_value(restored_model.optimizer.learning_rate, LEARNING_RATE)
# tf.keras.backend.set_value(restored_model)
history = restored_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=1,
    epochs=100,
    steps_per_epoch = 2000,
    validation_steps = 400,
    initial_epoch=EPOCH, 
    callbacks=[checkpoint]
)

Epoch 16/50
2000/2000 [==============================] - ETA: 0s - loss: 0.2581 - acc: 0.5706WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 400 batches). You may need to use the repeat() function when building your dataset.

Epoch 16: loss improved from inf to 0.25813, saving model to C:\Users\mdc20\best_model_21_class_lr@0.002-1666761586.9275644.hdf5
2000/2000 [==============================] - 1045s 522ms/step - loss: 0.2581 - acc: 0.5706 - val_loss: 0.2487 - val_acc: 0.5744
Epoch 17/50
2000/2000 [==============================] - ETA: 0s - loss: 0.2563 - acc: 0.5728
Epoch 17: loss improved from 0.25813 to 0.25633, saving model to C:\Users\mdc20\best_model_21_class_lr@0.002-1666761586.9275644.hdf5
2000/2000 [==============================] - 847s 423ms/step - loss: 0.2563 - acc: 0.5728
Epoch 18/50
2000/2000 [==============================] - ETA: 0s 

In [82]:
os.chdir('C:\\Users\\mdc20\\')
full_model.save('test_kernel_21_class.hdf5')

In [21]:
from plot_keras_history import show_history, plot_history
import os
os.chdir('C:\\Users\\mdc20\\')
print(os.getcwd())



plot_keras_history(history)

full_model.save('type_kernel.keras_model')

C:\Users\mdc20
Epoch 1/10
1991/1991 [==============================] - ETA: 0s - loss: 24.8112 - acc: 0.5546 - categorical_crossentropy: 24.8112
Epoch 1: acc improved from -inf to 0.55458, saving model to best_model.hdf5
1991/1991 [==============================] - 722s 362ms/step - loss: 24.8112 - acc: 0.5546 - categorical_crossentropy: 24.8112 - val_loss: 18.4811 - val_acc: 0.5966 - val_categorical_crossentropy: 18.4811
Epoch 2/10
1991/1991 [==============================] - ETA: 0s - loss: 26.5472 - acc: 0.5445 - categorical_crossentropy: 26.5472
Epoch 2: acc did not improve from 0.55458
1991/1991 [==============================] - 653s 328ms/step - loss: 26.5472 - acc: 0.5445 - categorical_crossentropy: 26.5472 - val_loss: 14.1841 - val_acc: 0.5125 - val_categorical_crossentropy: 14.1841
Epoch 3/10
1991/1991 [==============================] - ETA: 0s - loss: 34.2294 - acc: 0.5302 - categorical_crossentropy: 34.2294
Epoch 3: acc did not improve from 0.55458
1991/1991 [==============

KeyboardInterrupt: 

In [ ]:
history3 = full_model.fit(
    train_dataset, 
    validation_data = val_dataset,
    workers=1,
    epochs=20,
)

In [ ]:
print(history)

In [ ]:
# full_model.save('sparse_categorical_cross_entropy.h5')
full_model.save('39_acc.h5')

In [ ]:
from keras.models import load_model
model = load_model('cat_cross_entropy_16_classes.h5')

In [ ]:
from sklearn.metrics import confusion_matrix

x_val, y_val = next(val_dataset)
y_prediction = model.predict(x_val)
print(y_prediction[2])
print(y_val[2])
print(np.shape(y_prediction))
# preds = np.argmax(y_prediction, axis=1)
# print(preds)

k = 4
axis = 1 # Search for top k values in row
top_k_indx = np.argpartition(y_prediction, -4, axis=axis)[-4:].T
print(top_k_indx)
print(np.shape(top_k_indx))
# result = confusion_matrix(y_val, y_prediction, normalize='y_pred=')
# print(y_prediction)